In [8]:
import pickle
#from loadData import dataLoader
import tensorflow as tf
#from CalculateEER import SKGetEER as GetEER
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_curve, auc
from scipy import interp


In [2]:
data = pickle.load(open('../Data/RHUcleaned.pickle', 'rb'))

In [9]:
X = data['data']
Y = data['labels']

In [10]:
def onehot(x,numlabels):
    t = [0 for i in range(numlabels)]
    t[x-1] = 1
    return t
def GetEER(y_score, y_test):
    #y_score = y_score.get_label()
    y_test = np.array(y_test.get_label()).reshape(len(y_score), 1)
    n_classes = y_score.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    missRate = dict()
    #print(y_test)
    #print([np.argmax(x) for x in y for y in y_score])
    #y_test_ = [onehot(int(x[0]), 100) for x in y_test]
    y_score_ = [onehot(x,100) for x in y_score]
    #print(np.shape(y_score), np.shape(y_test))
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_,y_score)
        roc_auc[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    missRate["micro"] = 1 - tpr["micro"]
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    missRate["macro"] = 1 - tpr["macro"]
    #print(np.dtype(min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])))
    return 'EER', min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])

def GetEER_(y_score, y_test):
	n_classes = y_score.shape[1]
	fpr = dict()
	tpr = dict()
	roc_auc = dict()
	missRate = dict()
	for i in range(n_classes):
		fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
		roc_auc[i] = auc(fpr[i], tpr[i])
	fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
	missRate["micro"] = 1 - tpr["micro"]
	all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
	# Then interpolate all ROC curves at this points
	mean_tpr = np.zeros_like(all_fpr)
	for i in range(n_classes):
		mean_tpr += interp(all_fpr, fpr[i], tpr[i])
	mean_tpr /= n_classes
	fpr["macro"] = all_fpr
	tpr["macro"] = mean_tpr
	missRate["macro"] = 1 - tpr["macro"]
	return  100*min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])


In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [21]:
X = np.array(data['data'])
X = np.array(X).reshape(np.shape(X)[0], np.shape(X)[1])
y = np.array(data['labels'])
#y_train = [onehot(x,100) for x in train_data['labels']]
#X_test = np.array(test_data['features']).reshape(len(test_data['features']), 1, 28, 1)
#y_test = [onehot(x,100) for x in test_data['labels']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=12)
#X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=122)
print([len(x) for x in [X_test, y_test, X_train, y_train]])

#y_train=np.reshape(y_train,(len(y_train)*100))
#y_test = np.reshape(y_test, (len(y_test)*100))

[144, 144, 810, 810]


In [39]:
y_train_ = [np.argmax(x) for x in y_train]
y_test_ = [np.argmax(x) for x in y_test]
xg_train = xgb.DMatrix(X_train, label = y_train_)
xg_test = xgb.DMatrix(X_test, label =  y_test_)
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.1
param['max_depth'] = 9
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 52
param['n_estimators'] = 1000
param['nrounds'] = 200
param['tree_method']= 'hist'

n_classes = 52
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 200
bst = xgb.train(param, xg_train, num_round,  watchlist)#, feval = GetEER, maximize = False)



[0]	train-merror:0.402469	test-merror:0.638889
[1]	train-merror:0.230864	test-merror:0.569444
[2]	train-merror:0.140741	test-merror:0.534722
[3]	train-merror:0.112346	test-merror:0.506944
[4]	train-merror:0.090123	test-merror:0.5
[5]	train-merror:0.07037	test-merror:0.486111
[6]	train-merror:0.051852	test-merror:0.479167
[7]	train-merror:0.039506	test-merror:0.451389
[8]	train-merror:0.034568	test-merror:0.444444
[9]	train-merror:0.025926	test-merror:0.4375
[10]	train-merror:0.018519	test-merror:0.416667
[11]	train-merror:0.016049	test-merror:0.395833
[12]	train-merror:0.014815	test-merror:0.388889
[13]	train-merror:0.012346	test-merror:0.395833
[14]	train-merror:0.012346	test-merror:0.388889
[15]	train-merror:0.011111	test-merror:0.388889
[16]	train-merror:0.008642	test-merror:0.381944
[17]	train-merror:0.006173	test-merror:0.381944
[18]	train-merror:0.004938	test-merror:0.381944
[19]	train-merror:0.003704	test-merror:0.375
[20]	train-merror:0.003704	test-merror:0.368056
[21]	train-me

[197]	train-merror:0	test-merror:0.3125
[198]	train-merror:0	test-merror:0.3125
[199]	train-merror:0	test-merror:0.3125


In [37]:
pred = bst.predict(xg_test)
error_rate = np.sum(np.argmax(pred) != np.argmax(y_test))/np.shape(y_test)[0]

In [38]:
error_rate

0.006944444444444444

In [33]:
GetEER_(pred,y_test)

/usr/local/lib/python3.6/site-packages/sklearn/metrics/ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


0.0